In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

## Specifying each path

In [2]:
dataset = 'data/gestures_coordinates.csv'
model_save_path = 'files/model.hdf5'
tflite_save_path = 'files/tflite_save.tflite'

## Number of Classes (Gestures)

In [3]:
num_classes = 10

## Splitting Data

In [4]:
X = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=RANDOM_SEED)

## Building the Model

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
Total params: 1,180
Trainable params: 1,180
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [8]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

## Training the Model

In [9]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
7/9 [======================>.......] - ETA: 0s - loss: 2.4423 - accuracy: 0.1429 
Epoch 1: saving model to files\model.hdf5
9/9 [==============================] - 3s 114ms/step - loss: 2.4352 - accuracy: 0.1425 - val_loss: 2.3060 - val_accuracy: 0.1398
Epoch 2/1000
1/9 [==>...........................] - ETA: 0s - loss: 2.4163 - accuracy: 0.1172
Epoch 2: saving model to files\model.hdf5
9/9 [==============================] - 0s 16ms/step - loss: 2.3491 - accuracy: 0.1550 - val_loss: 2.2799 - val_accuracy: 0.1425
Epoch 3/1000
1/9 [==>...........................] - ETA: 0s - loss: 2.2786 - accuracy: 0.1328
Epoch 3: saving model to files\model.hdf5
9/9 [==============================] - 0s 19ms/step - loss: 2.3240 - accuracy: 0.1452 - val_loss: 2.2600 - val_accuracy: 0.1478
Epoch 4/1000
1/9 [==>...........................] - ETA: 0s - loss: 2.3151 - accuracy: 0.1484
Epoch 4: saving model to files\model.hdf5
9/9 [==============================] - 0s 26ms/step - loss: 2.2836 - a

In [10]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

3/3 [==============================] - 0s 6ms/step - loss: 0.4038 - accuracy: 0.9059


In [11]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [12]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 188ms/step
[2.4184088e-16 7.2406662e-01 7.5954234e-12 3.6881049e-12 4.1774184e-02
 2.6546579e-02 1.5603700e-07 5.9197730e-14 2.0761244e-01 4.0830681e-16]
1
